In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# https://www.kaggle.com/rounakbanik/the-movies-dataset 900mb. diğer tablolarla işimiz yok. indirmek istersen link
df = pd.read_csv("../input/movies-metadata/movies_metadata.csv",
                 low_memory=False)  # DtypeWarning kapamak icin



In [2]:
tfidf = TfidfVectorizer(stop_words="english")       # kullanacağımız metodu atama yöntemi ile hazırladık. 
#stopwords on, and, in gibi kelimelerin hesaba katılmamasını sağlar.

In [3]:
df[df['overview'].isnull()].shape  #kaç tane boş değer olduğunu görmek için bir yöntem

(954, 24)

In [4]:
df = df[~df["title"].duplicated(keep='last')]
df = df[~df["title"].isna()]
df = df[~df["overview"].isna()]

In [5]:
df['overview'] = df['overview'].fillna('') #NaN kelimesinin bizi yanıltmaması için boş değerler "" şeklinde tamamen boş bırakılır.

In [6]:
#tfidf adlı önceden atadığımız metodu çağırırız. fit ile içindeki bilgilere uygula deriz. 
#transform da eski değerleri yeni değerlerle kalıcı olarak değiştirir. 

tfidf_matrix = tfidf.fit_transform(df['overview'])


In [7]:
# Matrisin hafızadaki boyutunu yarıya indirme
tfidf_matrix = tfidf_matrix.astype(np.float32)

In [8]:
tfidf_matrix.shape  #satırdakiler film isimleri, sütundakiler kelimeler

(41344, 73955)

In [9]:
#tfidf.get_feature_names()  #değişken isimlerini görmeye yarayan bir komut

In [10]:
del df


In [11]:
# satırlarda açıklamlar, stunlarda terimler kesişiminde tf-idf skorları var.
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

**COSİNE SİM HESAPLAMA**
amacımız filmler arasındaki benzerlik ya da uzaklıkları bulmak
tfidf matrix'i oluşturuldukjtan sonra bunun üzerine öklit ya da cosine sim fonksiyonları kurularak, filmler arasındaki uzaklıklar belirlenir.

In [12]:
cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)

# Filtreleme
df = df[~df["title"].duplicated(keep='last')]
df = df[~df["title"].isna()]
df = df[~df["overview"].isna()]

#TFIDF Oluşturma
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Matrisin hafızadaki boyutunu yarıya indirme
tfidf_matrix = tfidf_matrix.astype(np.float32)

# Cosine Similarity hesaplaması
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [13]:
""" her satırda da sütunlarda da tüm filmer var. her saturda bir film. 
her sütunda, satırdaki filmin sütundaki filme ne derecede benzediği gösteriiyor.
"""
#cosine_sim.shape
#cosine_sim[1]

' her satırda da sütunlarda da tüm filmer var. her saturda bir film. \nher sütunda, satırdaki filmin sütundaki filme ne derecede benzediği gösteriiyor.\n'

Yukarıda cosine yöntemi ile tm filmlerin birbirine olan uzaklıklarını bulduk ancak film isimleri mevcut olmadığı için anlaşılır değil. Bununla ilgili olarak çözüm üreteceğiz.

In [14]:
indices = pd.Series(df.index, index=df['title'])
#indices adlı yeni bir pandas series oluşturuldu. index'e film isimleri koyuldu. yanında da hangi indexde bulundukları gösteriliyor.

NameError: name 'df' is not defined

In [ ]:
indices.index.value_counts() # bu kod ile bazı film isimlerinden birden fazla sayıda olduğunu görüyoruz. 

In [ ]:
#duplicated tekrar eden kodları seçer. indices serisinin indexlerinde tekrar edenleri bul. 
#sonuncuları hariç hepsini seç der. Bu seçtikelrin hariç hepsini (~) ata.
indices = indices[~indices.index.duplicated(keep='last')]

In [ ]:
#sherlock filminin indexi bir değere atanır.
movie_index = indices["Sherlock Holmes"]

In [ ]:
#cosine_sim matrixinde ilgili index'e giderek, filmin diğer filmlerle olan ilişkileri bulunur.
cosine_sim[movie_index]

In [ ]:
#yeni bir ps oluşturulur. index'de ikinci filmerin index numaraları, score'da benzerlik skorları.
similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"])

In [ ]:
similarity_scores.head()

In [ ]:
#oluşturulan serinin score'a göre değerleri azalacak şekilde sıralanır. 
ilk film kendisi olduu için ikinci sıradan itibaren ilk 10 film seçilir.bunların indexleri allınır
movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [ ]:
#DF'in title sütunundaki seçil indexler gösterilir. filmimize benzeyen ilk 10 film listeleniş olur.

In [ ]:
df['title'].iloc[movie_indices]

In [ ]:
"""
#################################
# 4. Çalışma Scriptinin Hazırlanması
#################################

def content_based_recommender(title, cosine_sim, dataframe):
    # index'leri olusturma
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # title'ın index'ini yakalama
    movie_index = indices[title]
    # title'a gore benzerlik skorlarını hesapalama
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    # kendisi haric ilk 10 filmi getirme
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]

content_based_recommender("Sherlock Holmes", cosine_sim, df)

content_based_recommender("The Matrix", cosine_sim, df)

content_based_recommender("The Godfather", cosine_sim, df)

content_based_recommender('The Dark Knight Rises', cosine_sim, df)


def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim


cosine_sim = calculate_cosine_sim(df)
content_based_recommender('The Dark Knight Rises', cosine_sim, df)
# 1 [90, 12, 23, 45, 67]
# 2 [90, 12, 23, 45, 67]
# 3
"""